In [4]:
%%capture
!pip install hummingbird-ml
!sudo pip install xgboost
!pip install optuna

In [5]:
import pickle
from os import listdir
from os.path import isfile, join
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score

from hummingbird.ml import convert,load

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
with open('drive/MyDrive/RealLifeViolencePickle/training_violence2.pickle', 'rb') as handle:
    training_run2 = pickle.load(handle)
with open('drive/MyDrive/RealLifeViolencePickle/training_violence3.pickle', 'rb') as handle:
    training_run3 = pickle.load(handle)
with open('drive/MyDrive/RealLifeViolencePickle/training_violence4.pickle', 'rb') as handle:
    training_run4 = pickle.load(handle)
with open('drive/MyDrive/RealLifeViolencePickle/valid_violence.pickle', 'rb') as handle:
    allTestingData = pickle.load(handle)

In [8]:
allTrainingData = np.concatenate([training_run2, training_run3, training_run4])

In [9]:
allTrainingData.shape

(73384, 3780)

In [10]:
allTestingData.shape

(18508, 3780)

In [11]:
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence2.pickle', 'rb') as labels:
  labels_run2 = pickle.load(labels)
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence3.pickle', 'rb') as labels:
  labels_run3 = pickle.load(labels)
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence4.pickle', 'rb') as labels:
  labels_run4 = pickle.load(labels)
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence_valid.pickle', 'rb') as labels:
  listOfFrameLabelsTest = pickle.load(labels)

In [12]:
listOfFrameLabels = np.concatenate([labels_run2, labels_run3, labels_run4])

In [13]:
framesTrain, labelsTrain = allTrainingData, listOfFrameLabels
framesTest, labelsTest = allTestingData, listOfFrameLabelsTest

## Decision Tree

In [ ]:
print("---------------Decision Tree---------------")
from sklearn.tree import DecisionTreeClassifier
clf_model = DecisionTreeClassifier(criterion="gini", random_state=42,max_depth=3, min_samples_leaf=5)   
clf_model.fit(framesTrain, labelsTrain)
labelPredict = clf_model.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('ROC-AUC Score : ', roc_auc_score(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

---------------Decision Tree---------------
[[8676   16]
 [ 899 8917]]
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      8692
           1       1.00      0.91      0.95      9816

    accuracy                           0.95     18508
   macro avg       0.95      0.95      0.95     18508
weighted avg       0.95      0.95      0.95     18508

ROC-AUC Score :  0.9532870299005615
Total Accuracy:  0.9505619191700886


In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# def objective(trial):
#     max_depth = trial.suggest_int("max_depth", 2, 612)
#     min_samples_split = trial.suggest_int("min_samples_split", 2, 612)
#     max_leaf_nodes = int(trial.suggest_int("max_leaf_nodes", 2, 612))
#     criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    
#     DTC = DecisionTreeClassifier(min_samples_split = min_samples_split, 
#                                 max_leaf_nodes = max_leaf_nodes,
#                                 criterion = criterion)
#     DTC.fit(framesTrain, labelsTrain)
#     return 1.0 - accuracy_score(labelsTest, DTC.predict(framesTest))

In [ ]:
with open('drive/MyDrive/EventDetection/saved_models/skl_decisiontree.pickle', 'wb')as saved:
    pickle.dump(clf_model, saved, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Use Hummingbird to convert the model to PyTorch
model = convert(clf_model, 'pytorch')
# Save the model
model.save('drive/MyDrive/EventDetection/saved_models/pytorch_decisiontree')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_features=2, max_depth=2, random_state=0)
gb_clf.fit(framesTrain, labelsTrain)
labelPredict = gb_clf.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('ROC-AUC Score : ', roc_auc_score(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

[[8692    0]
 [ 604 9212]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      8692
           1       1.00      0.94      0.97      9816

    accuracy                           0.97     18508
   macro avg       0.97      0.97      0.97     18508
weighted avg       0.97      0.97      0.97     18508

ROC-AUC Score :  0.9692339038304809
Total Accuracy:  0.9673654635833153


In [ ]:
with open('drive/MyDrive/EventDetection/saved_models/skl_gb.pickle', 'wb')as saved:
    pickle.dump(gb_clf, saved, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Use Hummingbird to convert the model to PyTorch
model = convert(gb_clf, 'pytorch')
# Save the model
model.save('drive/MyDrive/EventDetection/saved_models/pytorch_gb')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(framesTrain, labelsTrain)
labelPredict = clf.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('ROC-AUC Score : ', roc_auc_score(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

In [ ]:
with open('drive/MyDrive/EventDetection/saved_models/skl_randomforest.pickle', 'wb')as saved:
    pickle.dump(clf, saved, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Use Hummingbird to convert the model to PyTorch
model = convert(clf, 'pytorch')
# Save the model
model.save('drive/MyDrive/EventDetection/saved_models/pytorch_randomforest')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


## AdaBoost

In [14]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
ada_clf.fit(framesTrain, labelsTrain)
labelPredict = ada_clf.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('ROC-AUC Score : ', roc_auc_score(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

[[8596   96]
 [1047 8769]]
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      8692
           1       0.99      0.89      0.94      9816

    accuracy                           0.94     18508
   macro avg       0.94      0.94      0.94     18508
weighted avg       0.94      0.94      0.94     18508

ROC-AUC Score :  0.9411463847823419
Total Accuracy:  0.9382429219796845


In [15]:
with open('drive/MyDrive/EventDetection/saved_models/skl_adaboost.pickle', 'wb')as saved:
    pickle.dump(ada_clf, saved, protocol=pickle.HIGHEST_PROTOCOL)